In [ ]:
"""
TODOs:

visualization of layers to see how it trains: https://github.com/yosinski/deep-visualization-toolbox
a problem with the shuffling of the data: https://stanford.edu/~shervine/blog/keras-how-to-generate-data-on-the-fly
give weights to classes: #https://www.tensorflow.org/tutorials/structured_data/imbalanced_data --> it'll depend on the magnitude of the loss functions

"""

In [1]:
import os
from tensorflow.python.client import device_lib

def get_available_devices():
    local_device_protos = device_lib.list_local_devices()
    return [x.name for x in local_device_protos]

#Run from server

gpu4 = "GPU-71b4cdfc-e381-0b98-9b24-4fc06284b496" 
gpu5 = "GPU-99d0769a-9f86-4800-a40e-2320dddcf5d1" 
gpu6 = "GPU-7423cfb5-cff4-ec4d-7e96-ea6e1591d56f"
gpu7 = "GPU-c0a8738f-6dd0-1b78-c38f-4969fd3886a8"

os.environ["CUDA_VISIBLE_DEVICES"]= gpu6 + "," + gpu5

print(get_available_devices())

['/device:CPU:0', '/device:XLA_CPU:0', '/device:XLA_GPU:0', '/device:XLA_GPU:1', '/device:GPU:0', '/device:GPU:1']


In [2]:
import nibabel as nib
import matplotlib.pyplot as plt
import numpy as np
import h5py

In [3]:
import tensorflow as tf
from tensorflow.keras import backend as K
from tensorflow.keras.layers import Activation, Lambda, GlobalAveragePooling2D, concatenate
from tensorflow.keras.layers import UpSampling2D, Conv2D, Dropout, MaxPooling2D, Conv2DTranspose
from tensorflow.keras.layers import Dense, Flatten, Input
from tensorflow.keras.models import Model, Sequential, load_model
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from tensorflow.keras.optimizers import RMSprop, Adam, SGD

In [4]:
IMG_HEIGHT = 240
IMG_WIDTH = 240
IMG_DEPTH = 155
N_IMG = 369
length_file = IMG_DEPTH * N_IMG
h5py_file_name = 'training.hdf5'

___

Generate and create data arrays

In [ ]:
ImgDir = os.path.join("..","data","MICCAI_BraTS2020_TrainingData")
features_path = list()
labels_path = list()
count = 0
lim = 10

for folder in os.listdir(ImgDir):
    count +=1
    if 'Training' in folder:
        new_dir = os.path.join(ImgDir,folder)
        data = os.listdir(new_dir)
        for files in data:
            if 'flair' in files:
                features_path.append(os.path.join(new_dir, files))
            if 'seg' in files:
                labels_path.append(os.path.join(new_dir, files))

print(len(features_path))
print(len(labels_path))


In [ ]:
img_conc_features = np.zeros((len(features_path),IMG_HEIGHT,IMG_WIDTH,IMG_DEPTH))
path = features_path

for file, i in zip(path,range(len(path))):
    img = nib.load(file)
    imgarr = img.get_fdata()
    img_conc_features[i,:,:,:] = imgarr
    
img_conc_features = np.concatenate(img_conc_features,axis=2)
print(np.shape(img_conc_features))

In [ ]:
img_conc_labels = np.zeros((len(labels_path),IMG_HEIGHT,IMG_WIDTH,IMG_DEPTH))
path = labels_path

for file, i in zip(path,range(len(path))):
    img = nib.load(file)
    imgarr = img.get_fdata()
    img_conc_labels[i,:,:,:] = imgarr
    
img_conc_labels = np.concatenate(img_conc_labels,axis=2)
print(np.shape(img_conc_labels))

In [ ]:
with h5py.File(os.path.join('..','data',h5py_file_name), 'a') as f:
    f.create_dataset("features", data=img_conc_features, compression="gzip")
    f.create_dataset("labels", data=img_conc_labels, compression="gzip")

In [ ]:
hdf5_store_Brats = h5py.File(os.path.join('..','data',h5py_file_name), "r")
test_img = hdf5_store_Brats["features"][:, :, 100]
plt.imshow(test_img, cmap ='gray')
hdf5_store_Brats.close()

In [ ]:
hdf5_store_Brats = h5py.File(os.path.join('..','data',h5py_file_name), "r")
test_img = hdf5_store_Brats["labels"][:, :, 100]
plt.imshow(test_img, cmap ='gray')
hdf5_store_Brats.close()

___

Load data and model

In [5]:
# images = X_nib.get_fdata()
# labels = y_nib.get_fdata()

with h5py.File(os.path.join('..','data',h5py_file_name), "r") as f:
    images_train = f["features"][()]
    labels_train = f["labels"][()]

batch_size = 32
smooth = 0.005

In [6]:
#https://towardsdatascience.com/using-custom-building-blocks-in-tensorflow-2-0-550b88eb7aa2

class My_Custom_Generator(tf.keras.utils.Sequence):
  
    def __init__(self, images, labels, batch_size):
        self.images = images
        self.labels = labels
        self.batch_size = batch_size
        self.shuffle = True
        
    def on_epoch_end(self):
        self.indexes = np.arange(len(self.list_IDs))
        if self.shuffle == True:
            np.random.shuffle(self.indexes)

    def __len__(self) :
        return (np.ceil(len(self.images) / float(self.batch_size))).astype(np.int)


    def __getitem__(self, idx) :
        batch_x = self.images[:,:,idx * self.batch_size : (idx+1) * self.batch_size]
        batch_y = self.labels[:,:,idx * self.batch_size : (idx+1) * self.batch_size]
        #240,240,32
        batch_x /= 255.
        batch_y /= 255.

        train_image = []
        train_label = []
        
        #for i in range(0, len(batch_x)):
        for i in range(batch_size):

            X_new = np.zeros((240, 240 ,3), np.float32)
            y_new = np.zeros((240, 240 ,3), np.float32)

            X_new[:,:,0] = batch_x[:,:,i]
            X_new[:,:,1] = batch_x[:,:,i]
            X_new[:,:,2] = batch_x[:,:,i]

            y_new[:,:,0] = batch_y[:,:,i]
            y_new[:,:,1] = batch_y[:,:,i]
            y_new[:,:,2] = batch_y[:,:,i]

            train_image.append(X_new)
            train_label.append(y_new)

            #img_path = batch_x[i]
            #label = batch_y[i]
            #  # read method takes image path and label and returns corresponding matrices
            #image, label_matrix = read(img_path, label)
            #train_image.append(image)
            #train_label.append(label_matrix)
            
        #test channel creation done succesfully
        
        return np.array(train_image), np.array(train_label)

my_training_batch_generator = My_Custom_Generator(images_train, labels_train, batch_size)

In [7]:
class printbatch(tf.keras.callbacks.Callback):
    def on_train_batch_begin(self, batch, logs=None):
        keys = list(logs.keys())
        print("...Training: start of batch {}; got log keys: {}".format(batch, keys))

    def on_train_batch_end(self, batch, logs=None):
        keys = list(logs.keys())
        print("...Training: end of batch {}; got log keys: {}".format(batch, keys))

    #def on_epoch_end(self, epoch, logs=None):
    #    keys = list(logs.keys())
    #    
    #    print("End epoch {} of training; got log keys: {}".format(epoch, keys))

pb = printbatch()

In [8]:
def dice_coef(y_true, y_pred):
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    return (2. * intersection + smooth) / (K.sum(y_true_f) + K.sum(y_pred_f) + smooth)

def dice_coef_loss(y_true, y_pred):
    return (1-dice_coef(y_true, y_pred))

def get_model():
    in1 = Input(shape=(IMG_HEIGHT, IMG_WIDTH, 3))

    conv1 = Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(in1)
    conv1 = Dropout(0.2)(conv1)
    conv1 = Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(conv1)
    pool1 = MaxPooling2D((2, 2))(conv1)

    conv2 = Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(pool1)
    conv2 = Dropout(0.2)(conv2)
    conv2 = Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(conv2)
    pool2 = MaxPooling2D((2, 2))(conv2)

    conv3 = Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(pool2)
    conv3 = Dropout(0.2)(conv3)
    conv3 = Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(conv3)
    pool3 = MaxPooling2D((2, 2))(conv3)

    conv4 = Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(pool3)
    conv4 = Dropout(0.2)(conv4)
    conv4 = Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(conv4)

    up1 = concatenate([UpSampling2D((2, 2))(conv4), conv3], axis=-1)
    conv5 = Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(up1)
    conv5 = Dropout(0.2)(conv5)
    conv5 = Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(conv5)
    
    up2 = concatenate([UpSampling2D((2, 2))(conv5), conv2], axis=-1)
    conv6 = Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(up2)
    conv6 = Dropout(0.2)(conv6)
    conv6 = Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(conv6)

    up2 = concatenate([UpSampling2D((2, 2))(conv6), conv1], axis=-1)
    conv7 = Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(up2)
    conv7 = Dropout(0.2)(conv7)
    conv7 = Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(conv7)
    segmentation = Conv2D(3, (1, 1), activation='sigmoid', name='seg')(conv7)

    model = Model(inputs=[in1], outputs=[segmentation])   

    #model.compile(optimizer = 'adam',loss = 'categorical_crossentropy',metrics=['accuracy'])

    model.compile(optimizer=Adam(lr=1e-5, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.000000199), loss=dice_coef_loss, metrics=[dice_coef])

    return model

In [9]:
model2=get_model()
model2.summary()

Model: "functional_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 240, 240, 3) 0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 240, 240, 32) 896         input_1[0][0]                    
__________________________________________________________________________________________________
dropout (Dropout)               (None, 240, 240, 32) 0           conv2d[0][0]                     
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 240, 240, 32) 9248        dropout[0][0]                    
_______________________________________________________________________________________

In [10]:
def plot_metrics(history):
    metrics = ['loss', 'auc', 'precision', 'recall']
    for n, metric in enumerate(metrics):
        name = metric.replace("_"," ").capitalize()
        plt.subplot(2,2,n+1)
        plt.plot(history.epoch, history.history[metric], color=colors[0], label='Train')
        plt.plot(history.epoch, history.history['val_'+metric],
                 color=colors[0], linestyle="--", label='Val')
        plt.xlabel('Epoch')
        plt.ylabel(name)
        if metric == 'loss':
            plt.ylim([0, plt.ylim()[1]])
        elif metric == 'auc':
            plt.ylim([0.8,1])
        else:
            plt.ylim([0,1])

    plt.legend()

In [13]:
# modify the fit_generator call to include the callback pb
model2.fit(x = my_training_batch_generator, steps_per_epoch = int(len(images_train) // batch_size), epochs = 20, verbose=1, callbacks=[pb], validation_data=None, class_weight=None)

Epoch 1/20
...Training: start of batch 0; got log keys: []


ResourceExhaustedError:  OOM when allocating tensor with shape[32,32,240,240] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc
	 [[node functional_1/conv2d/Relu (defined at <ipython-input-11-92a55fcbd316>:2) ]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.
 [Op:__inference_train_function_2372]

Function call stack:
train_function


In [ ]:
plot_metrics(training_history)

___

In [15]:
import SimpleITK as sitk
img_flair = sitk.ReadImage(os.path.join("..", "data","MICCAI_BraTS2020_TrainingData","BraTS20_Training_001","BraTS20_Training_001_flair.nii.gz"))
nda_flair = sitk.GetArrayViewFromImage(img_flair)
print(np.shape(nda_flair))

for i in range(len(nda_flair[:,1,1])):

    X_new = np.zeros((240, 240 ,3), np.float32)

    X_new[:,:,0] = batch_x[:,:,i]
    X_new[:,:,1] = batch_x[:,:,i]
    X_new[:,:,2] = batch_x[:,:,i]

    train_image.append(X_new)

(155, 240, 240)


NameError: name 'batch_x' is not defined

In [ ]:
def generator():
    
    data_length = len(X_nib.get_fdata()[1,1,:])
    chunks = np.floor(data_length / batch_size)
    print('A')
    X = X_nib.get_fdata()
    y = y_nib.get_fdata()
    print('B')
    X /= 255
    y /= 255
    print('C')
    
    while True:
        
        for i in range(int(chunks)):
            X_yield = []
            y_yield = []

            X_mod = X[:,:,i*32:(i+1)*32]
            y_mod = y[:,:,i*32:(i+1)*32]
            print(np.shape(X_mod))

            for j in range(32):
                X_new = np.zeros((240, 240 ,3), np.uint8)
                X_new[:,:,0] = X_mod[:,:,j]
                X_new[:,:,1] = X_mod[:,:,j]
                X_new[:,:,2] = X_mod[:,:,j]

                X_yield.append(X_new)
                print(np.shape(X_yield))

                y_new = np.zeros((240, 240 ,3), np.uint8)
                y_new[:,:,0] = y_mod[:,:,j]
                y_new[:,:,1] = y_mod[:,:,j]
                y_new[:,:,2] = y_mod[:,:,j]

                y_yield.append(y_new)

            yield np.reshape(X_yield,(32,240,240,3)), np.reshape(y_yield,(32,240,240,3))
        

my_generator = generator()